# Clase MLP

In [20]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Función de activación sigmoide
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivada de la sigmoide
def sigmoid_derivative(x):
    return x * (1 - x)

# Establece la semilla para la generación de números aleatorios
def seed(random_state=33):
    np.random.seed(random_state)

# Inicialización de javier
def xavier_initialization(input_size, output_size):
    # ¿En el parametro size es output, input?
    return np.random.normal(scale=np.sqrt(1.0 / input_size), size=(output_size, input_size))

# Inicialización normal
def normal_initialization(input_size, output_size):
    return np.random.randn(output_size, input_size) * 0.1

# Preprocesado de datos
def preprocesar(ruta):
    datos = pd.read_csv(ruta, header=0)
    datos_crudos = datos.to_numpy()

    x = datos_crudos[:, :-1]
    y = datos_crudos[:, -1:]

    return x, y

# Normalizar los datos
def normalizar_datos(X):
    scaler = StandardScaler()
    return scaler.fit_transform(X)

# Crear mini lotes
def create_minibatches(X, y, batch_size):
    n_samples = X.shape[0]
    indices = np.random.permutation(n_samples)
    X_shuffled, y_shuffled = X[indices], y[indices]
    
    n_batches = int(np.ceil(n_samples / batch_size))
    for i in range(n_batches):
        start = i * batch_size
        end = min((i + 1) * batch_size, n_samples)
        yield X_shuffled[start:end], y_shuffled[start:end]

# Probar modelo
def evaluar_modelo_prueba(modelo, ruta_prueba, normalizar):
    x_test_crudo, Y_test_crudo = preprocesar(ruta_prueba)

    if normalizar:
        x_test = normalizar_datos(x_test_crudo)
    else:
        x_test = x_test_crudo
    
    metricas = modelo.evaluar(x_test, Y_test_crudo)

    return metricas

def calcular_metricas_completas(y_real, y_pred):
    """
    Calcula todas las métricas requeridas
    """
    y_real_flat = y_real.reshape(-1)
    y_pred_flat = y_pred.reshape(-1)
    
    accuracy = np.mean(y_real_flat == y_pred_flat)
    precision = precision_score(y_real_flat, y_pred_flat, zero_division=0)
    recall = recall_score(y_real_flat, y_pred_flat, zero_division=0)
    f1 = f1_score(y_real_flat, y_pred_flat, zero_division=0)
    
    return {
        'exactitud': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1)
    }


class MLP_TODO:
    def __init__(self, num_entradas, num_neuronas_ocultas, num_salidas, epochs, batch_size=128, learning_rate=0.2, random_state=42, initialization="xavier"):

        # Construcción
        seed(random_state)
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        
        self.error_mse = []
        self.accuracy_epoca = []
        self.f1_score_epoca = []
        
        # definir las capas
        if initialization == 'xavier':
            init_fun = xavier_initialization
        else : 
            init_fun = normal_initialization

        self.W1 = init_fun(num_entradas, num_neuronas_ocultas)
        self.b1 = np.zeros((1, num_neuronas_ocultas))
        self.W2 = init_fun(num_neuronas_ocultas, num_salidas)
        self.b2 = np.zeros((1, num_salidas))

        self.metricas_entrenamiento = []  # Métricas por época
        self.metricas_prueba = []  # Métricas por época en conjunto de prueba

    def forward(self, X):
        #----------------------------------------------
        # 1. Propagación hacia adelante (Forward pass)
        #----------------------------------------------
        self.X = X
        self.z_c1 = X @ self.W1.T + self.b1
        self.a_c1 = sigmoid(self.z_c1)
        self.z_c2 = self.a_c1 @ self.W2.T + self.b2
        y_pred = sigmoid(self.z_c2)  # Y^
        return y_pred

    def loss_function_MSE(self, y_pred, y):
        #----------------------------------------------
        # 2. Cálculo del error con MSE
        #----------------------------------------------
        self.y_pred = y_pred
        self.y = y
        error = np.mean((y_pred - y) ** 2)
        return error
    
    def backward(self):
        #----------------------------------------------
        # 3. Propagación hacia atrás (Backward pass)
        #----------------------------------------------
        
        #----------------------------------------------
        # Gradiente de la salida
        #----------------------------------------------
        dE_dy_pred = (self.y_pred - self.y) / self.y.shape[0] # Derivada del error respecto a la predicción con  N ejemplos
        d_y_pred_d_zc2 = sigmoid_derivative(self.y_pred)
        delta_c2 = dE_dy_pred * d_y_pred_d_zc2

        #----------------------------------------------
        # Gradiente en la capa oculta
        #----------------------------------------------
        # calcular la derivada de las suma ponderada respecto a las activaciones de la capa 1
        delta_c1 = (delta_c2 @ self.W2) * sigmoid_derivative(self.a_c1)

        #calcula el gradiente de pesos y bias
        self.dE_dW2 = delta_c2.T @ self.a_c1
        self.dE_db2 = np.sum(delta_c2, axis=0, keepdims=True)
        self.dE_dW1 = delta_c1.T @ self.X
        self.dE_db1 = np.sum(delta_c1, axis=0, keepdims=True)

    def update(self):  # Ejecución de la actualización de paramámetros
        #----------------------------------------------
        # Actualización de pesos de la capa de salida
        #---------------------------------------------- 
        
        self.W2 = self.W2 - self.learning_rate * self.dE_dW2 # Ojito con la T
        self.b2 = self.b2 - self.learning_rate * self.dE_db2

        #----------------------------------------------
        # Actuailzación de pesos de la capa oculta
        #----------------------------------------------
        #calcula el gradiente de la función de error respecto a los pesos de la capa 1
        self.W1 = self.W1 - self.learning_rate * self.dE_dW1
        self.b1 = self.b1 - self.learning_rate * self.dE_db1

    def predict(self, X):  # Predecir la categoría para datos nuevos
        y_pred = self.forward(X)
        # Obtener la clase para el clasificador binario
        y_pred = np.where(y_pred >= 0.5, 1, 0)
        return y_pred

    def train(self, X, Y):
        for epoch in range(self.epochs):

            num_batch = 0
            epoch_error  = 0

            # Procesamiento por lotes
            for X_batch, y_batch in create_minibatches(X, Y, self.batch_size):
                y_pred = self.forward(X_batch)
                error = self.loss_function_MSE(y_pred, y_batch)

                epoch_error += error
                self.backward() # cálculo de los gradientes
                self.update() # actualización de los pesos y bias
                num_batch += 1
            
            # Almacena el error promedio por época
            self.error_mse.append(epoch_error/num_batch)

            # Obtener predicciones binarias para todo el conjunto de entrenamiento
            y_pred_total = self.predict(X)

            # Calcular métricas completas en entrenamiento
            metricas_epoch = calcular_metricas_completas(Y, y_pred_total)
            metricas_epoch['mse'] = float(epoch_error/num_batch)
            self.metricas_entrenamiento.append(metricas_epoch)

            f1_score_epoca = metricas_epoch.get('f1_score', 0.0) 
            self.f1_score_epoca.append(f1_score_epoca)

            # Calcular la exactitud
            exactitud = self.calcular_accuracy(y_pred_total, Y) 
            
            # Almacenar la exactitud de la época
            self.accuracy_epoca.append(exactitud)

    def graficar(self, graficar_exactitud=False, graficar_f1_score=True, guardar=True, nombre="grafica"):
        """ 
        Grafica la curva de aprendizaje (MSE y opcionalmente F1-Score o Exactitud).
        """
        import matplotlib.pyplot as plt
        import numpy as np
        
        # Preparar datos
        mse = np.arange(len(self.error_mse))

        # Crear tabla
        plt.figure(figsize=(10,6))

        #Graficar MSE
        plt.plot(mse, self.error_mse, label="MSE", color="#e7b40d", linewidth=2)
        
        titulo = "Evolución del Error (MSE) durante el entrenamiento"

        """ 
        Para el F1-Score (PRIORIDAD)
        """
        if graficar_f1_score and len(self.f1_score_epoca) > 0:
            f1 = np.arange(len(self.f1_score_epoca))
            plt.plot(f1, self.f1_score_epoca, label="F1-Score", color="#0ac26f", linewidth=2)
            plt.ylabel("MSE / F1-Score")
            titulo = "Evolución del Error (MSE) y F1-Score durante el entrenamiento"

            """ 
            Para la exactitud (SOLO si F1-Score no está activo)
            """
        elif graficar_exactitud and len(self.accuracy_epoca) > 0:
            accuracy = np.arange(len(self.accuracy_epoca))
            plt.plot(accuracy, self.accuracy_epoca, label="Exactitud", color="#1f77b4", linewidth=2)
            plt.ylabel("MSE / Exactitud")
            titulo = "Evolución del Error (MSE) y Exactitud durante el entrenamiento"
        
        # Si no se grafica F1 ni Exactitud, el título queda el default (solo MSE)

        plt.title(titulo)
        plt.xlabel("Época")
        plt.legend()
        plt.grid(True, alpha=0.3)

        if guardar:
            plt.savefig(f'./Resultados/Graficas/{nombre}.svg')
        plt.show()

    def calcular_accuracy(self, y_pred, y_verdadera):
        y_verdadera_flat = y_verdadera.reshape(-1)
        y_pred_flat = y_pred.reshape(-1)
        return np.mean(y_verdadera_flat == y_pred_flat)

    def analizar(self, X, y):
        # Gráficar
        self.graficar(guardar=True, graficar_exactitud=False)

        # Valores reales y predicción
        y_pred = self.predict(X)
        print(f"valores reales: {y.flatten()}")
        print(f"Predicciones  : {y_pred.flatten()}")

        # Calcular exactitud
        exactitud = self.calcular_accuracy(y_pred, y)
        print(f"Exactitud: {exactitud}")

    def evaluar(self, x_test, y_test):
        y_gorrito = self.predict(x_test)

        accuracy = self.calcular_accuracy(y_gorrito, y_test)

        probabilidad = self.forward(x_test)
        mse = self.loss_function_MSE(probabilidad, y_test)

        metricas = {
            "Exactitud": accuracy,
            "mse": mse
        }

        return metricas

# Abir un archivo

In [21]:
import pandas as pd

# Definir las rutas de los datasets
rutas = {"spanish": {
    "train": "./Recursos/hateval_es_train.json",
    "test": "./Recursos/hateval_es_test.json",
    "all": "./Recursos/hateval_es_all.json"
},
"english": {
    "train": "./Recursos/hateval_en_train.json",
    "test": "./Recursos/hateval_en_test.json",
    "all": "./Recursos/hateval_en_all.json"
    }
}

# Aquí se almacenan todos los datasets
datos = {}

# Cargar todos los datasets
for idioma, archivos in rutas.items():
    datos[idioma] = {}
    for tipo, ruta in archivos.items():
        datos[idioma][tipo] = pd.read_json(ruta, lines=True)
        print(f"Cargando {idioma}_{tipo}, {len(datos[idioma][tipo])} datos")

print(datos["spanish"]["test"].head(10))

Cargando spanish_train, 4500 datos
Cargando spanish_test, 500 datos
Cargando spanish_all, 5000 datos
Cargando english_train, 9000 datos
Cargando english_test, 1000 datos
Cargando english_all, 10000 datos
      id  klass                                               text
0  20005      0  Me estoy comiendo la picada árabe más rica de ...
1  20006      1      @Haryachyzaychyk Callate zorra y mama duro! 😍
2  20011      0  Acabo de escuchar a Casado diciendo que hay DE...
3  20019      1  Y NADIE SE HA PREGUNTADO LO QUE LE VA A COSTAR...
4  20033      1  @Fed_Durand Callate come sobra, más zorra son ...
5  20039      0   te quiero hacer mi reina árabe bomboncitocaramel
6  20046      1            @andreaacata cállate perra JAJAJAJAJAJA
7  20047      0  En ""La Reina del Sur"" de @perezreverte , el ...
8  20063      1  @AnderssonBoscan Vieja zorra imbécil y MENTIRO...
9  20101      0  @rjimenez_perez tu eres un hijo de puta perver...


# Normalización de textos

In [22]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/jair/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.stem import SnowballStemmer

# Procesar un dataset aplicando noramalización y stemming
def preprocesar_dataset(df, idioma="spanish"):
    # Segun el idioma
    stemmer = SnowballStemmer(idioma)

    # obtener stopword segun idioma
    if idioma == "spanish":
        stop_words = set(stopwords.words('spanish'))
    else:  # english
        stop_words = set(stopwords.words('english'))

    def aplicar_stemming_y_eliminar_stopwords(texto):
        if pd.isna(texto): 
            return texto
        
        palabras = str(texto).split()
        # Filtrar stopwords y aplicar stemming
        palabras_filtradas = [
            stemmer.stem(palabra) 
            for palabra in palabras 
            if palabra.lower() not in stop_words
        ]
        return ' '.join(palabras_filtradas)
    
    df["text"] = df["text"].apply(normaliza_texto)
    df["text"] = df["text"].apply(aplicar_stemming_y_eliminar_stopwords)

    return df

In [24]:
# Normalización del texto

import unicodedata
import re
PUNCTUACTION = ";:,.\\-\"'/"
SYMBOLS = "()[]¿?¡!{}~<>|"
NUMBERS= "0123456789"
SKIP_SYMBOLS = set(PUNCTUACTION + SYMBOLS)
SKIP_SYMBOLS_AND_SPACES = set(PUNCTUACTION + SYMBOLS + '\t\n\r ')

def normaliza_texto(input_str,
                    punct=False,
                    accents=False,
                    num=False,
                    max_dup=2):
    """
        punct=False (elimina la puntuación, True deja intacta la puntuación)
        accents=False (elimina los acentos, True deja intactos los acentos)
        num= False (elimina los números, True deja intactos los acentos)
        max_dup=2 (número máximo de símbolos duplicados de forma consecutiva, rrrrr => rr)
    """
    
    nfkd_f = unicodedata.normalize('NFKD', input_str)
    n_str = []
    c_prev = ''
    cc_prev = 0
    for c in nfkd_f:
        if not num:
            if c in NUMBERS:
                continue
        if not punct:
            if c in SKIP_SYMBOLS:
                continue
        if not accents and unicodedata.combining(c):
            continue
        if c_prev == c:
            cc_prev += 1
            if cc_prev >= max_dup:
                continue
        else:
            cc_prev = 0
        n_str.append(c)
        c_prev = c
    texto = unicodedata.normalize('NFKD', "".join(n_str))
    texto = re.sub(r'(\s)+', r' ', texto.strip(), flags=re.IGNORECASE)
    return texto



# Aplicar normalización a los datasets

In [25]:
datos_preprocesados = {}

for idioma in datos:
    datos_preprocesados[idioma] = {}

    for tipo in datos[idioma]:
        datos_preprocesados[idioma][tipo] = preprocesar_dataset(datos[idioma][tipo], idioma)

print(datos_preprocesados["spanish"]["test"].head(10))

      id  klass                                               text
0  20005      0                           com pic arab mas ric vid
1  20006      1             @haryachyzaychyk callat zorr mam dur 😍
2  20011      0  acab escuch cas dic decen millon subsaharian a...
3  20019      1  nadi pregunt va cost hombr guantaz situacion s...
4  20033      1               @fed_durand callat com sobr mas zorr
5  20039      0              quier hac rein arab bomboncitocaramel
6  20046      1                @andreaacat callat perr jajajajajaj
7  20047      0  rein sur @perezrevert personaj guer davil tamb...
8  20063      1  @anderssonbosc viej zorr imbecil mentir cre so...
9  20101      0  @rjimenez_perez hij put pervert pedofil lacr d...


# Documento-Término

In [26]:
# Construye la matriz Documento-Término (num_documentos x tamaño_vocabulario) por frecuencia de aparición

def documento_termino(documentos, vocabulario):
    num_documentos = len(documentos)
    tamaño_vocabulario = len(vocabulario)
    matriz_frecuencias = np.zeros((num_documentos, tamaño_vocabulario), dtype=int)
    
    # Crear diccionario para búsqueda más cómoda
    indice_terminos = {termino: indice for indice, termino in enumerate(vocabulario)}
    
    terminos_desconocidos = set()
    
    for indice_documento, texto_documento in enumerate(documentos):
        # Dividir el documento en términos individuales
        terminos_documento = texto_documento.lower().split()
        
        for termino_actual in terminos_documento:
            if termino_actual in indice_terminos:
                indice_termino = indice_terminos[termino_actual]
                matriz_frecuencias[indice_documento, indice_termino] += 1
            else:
                terminos_desconocidos.add(termino_actual)
    
    # Reportar términos fuera del vocabulario si los hay
    if terminos_desconocidos:
        print(f"Advertencia: {len(terminos_desconocidos)} términos no encontrados en el vocabulario")
        # Opcional: mostrar algunos ejemplos de términos desconocidos
        # print(f"Ejemplos de términos desconocidos: {list(terminos_desconocidos)[:5]}")
    
    return matriz_frecuencias


# Construir el vocabulario único a partir de los textos procesados
def construir_vocabulario(datos_procesados, idioma, tipo="train"):
    textos = datos_preprocesados[idioma][tipo]["text"]
    
    palabras = []
    for texto in textos:
        palabras.extend(texto.lower().split())
    
    vocabulario = sorted(set(palabras))
    print(f"Vocabulario: {idioma}-{tipo}: {len(vocabulario)} términos únicos")
    return vocabulario

In [27]:
matrices_vsm = {}
vocabularios = {}
for idioma in datos_preprocesados:
    vocabularios[idioma] = construir_vocabulario(datos_preprocesados, idioma, "train")

for idioma in datos_preprocesados:
    matrices_vsm[idioma] = {}  # CORRECCIÓN: inicializar por idioma
    
    print(f"\nProcesando: {idioma}")
    
    for tipo in datos_preprocesados[idioma]:
        textos = datos_preprocesados[idioma][tipo]["text"].to_list()
        
        vsm = documento_termino(textos, vocabularios[idioma])
        
        matrices_vsm[idioma][tipo] = vsm
        print(f"Matriz {idioma}_{tipo}: {vsm.shape}")

Vocabulario: spanish-train: 13606 términos únicos
Vocabulario: english-train: 23153 términos únicos

Procesando: spanish
Matriz spanish_train: (4500, 13606)
Advertencia: 1074 términos no encontrados en el vocabulario
Matriz spanish_test: (500, 13606)
Advertencia: 1074 términos no encontrados en el vocabulario
Matriz spanish_all: (5000, 13606)

Procesando: english
Matriz english_train: (9000, 23153)
Advertencia: 1928 términos no encontrados en el vocabulario
Matriz english_test: (1000, 23153)
Advertencia: 1928 términos no encontrados en el vocabulario
Matriz english_all: (10000, 23153)


In [28]:
print(matrices_vsm["spanish"]["train"])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Funcion de Entrenamiento por lotes

In [29]:
import json
import os
import time
from datetime import datetime

def entrenar_modelos_idioma(idioma, configuraciones, archivo_resultados="resultados_entrenamiento.json"):
    """
    Entrena modelos MLP para un idioma específico con diferentes configuraciones
    
    Args:
        idioma (str): 'spanish' o 'english'
        configuraciones (list): Lista de diccionarios con configuraciones de hiperparámetros
        archivo_resultados (str): Ruta donde guardar los resultados
    
    Returns:
        list: Lista de modelos entrenados y sus métricas
    """
    
    # Crear directorio para resultados si no existe
    os.makedirs(os.path.dirname(archivo_resultados), exist_ok=True)
    
    # Obtener datos de entrenamiento y prueba
    X_train = matrices_vsm[idioma]["train"]
    y_train = datos_preprocesados[idioma]["train"]["klass"].values.reshape(-1, 1)
    X_test = matrices_vsm[idioma]["test"]
    y_test = datos_preprocesados[idioma]["test"]["klass"].values.reshape(-1, 1)
    
    print(f"Entrenando modelos para: {idioma}")
    print(f"Datos - Entrenamiento: {X_train.shape}, Prueba: {X_test.shape}")
    print(f"Total de configuraciones a probar: {len(configuraciones)}")
    
    resultados = []
    modelos_entrenados = []
    
    for i, config in enumerate(configuraciones, 1):
        print(f"\n--- Modelo {i}/{len(configuraciones)} ---")
        print(f"Configuración: {config}")
        
        try:
            # Crear y entrenar modelo
            modelo = MLP_TODO(
                num_entradas=X_train.shape[1],
                num_neuronas_ocultas=config['neuronas_ocultas'],
                num_salidas=1,
                epochs=config['epochs'],
                batch_size=config['batch_size'],
                learning_rate=config['learning_rate'],
                initialization=config.get('initialization', 'xavier')
            )
            
            print("Entrenando modelo...")
            start_time = time.time()
            modelo.train(X_train, y_train)
            training_time = time.time() - start_time
            
            # Evaluar en datos de prueba
            y_pred_test = modelo.predict(X_test)
            metricas_test = calcular_metricas_completas(y_test, y_pred_test)
            
            # Evaluar en datos de entrenamiento
            y_pred_train = modelo.predict(X_train)
            metricas_train = calcular_metricas_completas(y_train, y_pred_train)
            
            # Obtener métricas finales de entrenamiento
            mse_final = modelo.error_mse[-1] if modelo.error_mse else 0
            accuracy_final = modelo.accuracy_epoca[-1] if modelo.accuracy_epoca else 0
            
            # Guardar resultados
            resultado = {
                'id_configuracion': i,
                'idioma': idioma,
                'configuracion': config,
                'metricas_prueba': metricas_test,
                'metricas_entrenamiento': metricas_train,
                'mse_final': float(mse_final),
                'accuracy_final': float(accuracy_final),
                'tiempo_entrenamiento': training_time,
                'timestamp': datetime.now().isoformat(),
                'dimensiones_entrada': X_train.shape[1],
                'dimensiones_salida': 1
            }
            
            resultados.append(resultado)
            modelos_entrenados.append({
                'modelo': modelo,
                'configuracion': config,
                'resultado': resultado
            })
            
            print(f"✓ Entrenamiento completado en {training_time:.2f} segundos")
            print(f"  Exactitud prueba: {metricas_test['exactitud']:.4f}")
            print(f"  F1-score prueba: {metricas_test['f1_score']:.4f}")
            print(f"  MSE final: {mse_final:.6f}")
            
            # Guardar resultados parciales cada 10 modelos
            guardar_resultados_parciales(resultados, archivo_resultados, f"parcial_{i}")
                
        except Exception as e:
            print(f"✗ Error entrenando modelo {i}: {str(e)}")
            resultado_error = {
                'id_configuracion': i,
                'idioma': idioma,
                'configuracion': config,
                'error': str(e),
                'timestamp': datetime.now().isoformat()
            }
            resultados.append(resultado_error)
    
    # Guardar resultados finales
    guardar_resultados_completos(resultados, archivo_resultados)
    
    print(f"\n{'='*60}")
    print(f"Entrenamiento completado para {idioma}")
    print(f"Modelos exitosos: {len([r for r in resultados if 'error' not in r])}/{len(configuraciones)}")
    print(f"Resultados guardados en: {archivo_resultados}")
    
    return modelos_entrenados

def guardar_resultados_parciales(resultados, archivo, sufijo):
    """Guarda resultados parciales durante el entrenamiento"""
    nombre_archivo = archivo.replace('.json', f'_{sufijo}.json')
    print(f"Guardado en {nombre_archivo}")
    with open(nombre_archivo, 'w', encoding='utf-8') as f:
        json.dump(resultados, f, indent=2, ensure_ascii=False)


def guardar_resultados_completos(resultados, archivo):
    """Guarda todos los resultados en un archivo JSON"""
    resultados_completos = {
        'metadata': {
            'fecha_creacion': datetime.now().isoformat(),
            'total_configuraciones': len(resultados),
            'configuraciones_exitosas': len([r for r in resultados if 'error' not in r]),
            'configuraciones_fallidas': len([r for r in resultados if 'error' in r])
        },
        'resultados': resultados
    }
    
    with open(archivo, 'w', encoding='utf-8') as f:
        json.dump(resultados_completos, f, indent=2, ensure_ascii=False)

# Configuraciones de modelos a probar

In [30]:
def generar_configuraciones_completas():
    """Genera todas las configuraciones posibles para experimentos"""
    configuraciones = []
    
    # Definir rangos de hiperparámetros
    neuronas_ocultas_options = [32, 64, 128, 256]
    learning_rates = [0.01, 0.1]
    batch_sizes = [16, 32, 64]
    initializations = ['xavier', 'normal']
    epochs_options = [100]
    pesado_terminos_options = ['tf', 'tf-idf']
    ngram_options = ['unigramas', 'bigramas']
    
    # Generar todas las combinaciones
    for neuronas in neuronas_ocultas_options:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                for init in initializations:
                    for epochs in epochs_options:
                        for pesado in pesado_terminos_options:
                            for ngram in ngram_options:
                                config = {
                                    'neuronas_ocultas': neuronas,
                                    'epochs': epochs,
                                    'batch_size': batch_size,
                                    'learning_rate': lr,
                                    'initialization': init,
                                    'pesado_terminos': pesado,
                                    'ngramas': ngram
                                }
                                configuraciones.append(config)
    
    print(f"Total de configuraciones generadas: {len(configuraciones)}")
    return configuraciones

configuraciones_mlp = generar_configuraciones_completas()
config = configuraciones_mlp[::3]
print(config[60])

Total de configuraciones generadas: 192
{'neuronas_ocultas': 256, 'epochs': 100, 'batch_size': 32, 'learning_rate': 0.1, 'initialization': 'normal', 'pesado_terminos': 'tf', 'ngramas': 'unigramas'}


# Entrenamiento

In [31]:
# Entrenar modelos para español
#modelos_espanol = entrenar_modelos_idioma('spanish', config, "./Resultados/modelos_espanol_100_epoch.json")

# Entrenar modelos para inglés 
#modelos_ingles = entrenar_modelos_idioma('english', configuraciones_mlp, "./Resultados/modelos_ingles_100_epoch.json")


# Mejores resultados

In [42]:
import pandas as pd
import json

# 1. Cargar el JSON
# Si tu archivo JSON tiene la estructura que compartiste (una lista con un objeto principal),
# es mejor leerlo primero con el módulo 'json' para acceder a la clave anidada.
ruta_archivo = "./Resultados/modelos_espanol_100_epoch.json"

with open(ruta_archivo, 'r') as f:
    data = json.load(f)

# La lista de resultados está en el primer (y único) elemento, bajo la clave 'resultados'
resultados = data[0]['resultados']

# 2. Normalizar la estructura anidada en un DataFrame plano
# Usamos json_normalize para aplanar las claves anidadas como 'configuracion' y 'metricas_prueba'
res_df = pd.json_normalize(
    resultados,
    # Separa las claves anidadas con un punto (por defecto)
    sep='.' 
)

# 3. Ordenar el DataFrame por la métrica deseada
# Ordenamos por 'metricas_prueba.f1_score' de forma descendente (los mejores arriba)
columna_orden = 'metricas_prueba.f1_score'
res_df_ordenado = res_df.sort_values(
    by=columna_orden,
    ascending=False # False = de mayor a menor
)

# 4. Mostrar el encabezado de los mejores resultados
# print(f"--- DataFrame Ordenado por {columna_orden} (Top 5) ---")
# print(res_df_ordenado.head())

mejores_5_df = res_df_ordenado.head(5).copy()
furiosos_5 = pd.DataFrame()

furiosos_5['ID'] = mejores_5_df['id_configuracion']
furiosos_5['f1_score'] = mejores_5_df['metricas_prueba.f1_score']

furiosos_5['Configuracion'] = [
    {
        "num_neuronas_ocultas": row['configuracion.neuronas_ocultas'],
        "learning_rate": row['configuracion.learning_rate'],
        "batch_size": row['configuracion.batch_size'],
        "epochs": row['configuracion.epochs'],
        "inicializacion": row['configuracion.initialization']
    }
    for _, row in mejores_5_df.iterrows()
]

print("--- DataFrame 'furiosos_5' listo para entrenamiento ---")
print(furiosos_5)

--- DataFrame 'furiosos_5' listo para entrenamiento ---
    ID  f1_score                                      Configuracion
60  61  0.729258  {'num_neuronas_ocultas': 128, 'learning_rate':...
61  62  0.729258  {'num_neuronas_ocultas': 128, 'learning_rate':...
62  63  0.727273  {'num_neuronas_ocultas': 128, 'learning_rate':...
63  64  0.727273  {'num_neuronas_ocultas': 128, 'learning_rate':...
64  65  0.723112  {'num_neuronas_ocultas': 128, 'learning_rate':...


In [ ]:
# --- PASO 1: PREPARACIÓN DE DATOS (Asegúrate que X_train_es esté definido) ---

# Usaremos la matriz VSM de español y sus etiquetas.
# Se asume que X_train_es e Y_train_es están correctamente definidas
# (por ejemplo, con TF-IDF, o TF si no se implementó TF-IDF)

# Definición de datos de entrada (Ejemplo usando TF)
try:
    X_train_es = matrices_vsm["spanish"]["train"]
    Y_train_es = datos_preprocesados["spanish"]["train"]["klass"].values.reshape(-1, 1)
except NameError:
    print("¡Advertencia! Las variables 'matrices_vsm' y/o 'datos_preprocesados' no están definidas.")
    print("Asegúrate de ejecutar las celdas de carga y preparación de datos de tu notebook.")
    # Si las variables no existen, el código fallará aquí.

# --- PASO 2: ENTRENAMIENTO DE MODELOS ---

lista_modelos_entrenados = []
num_entradas = X_train_es.shape[1]

print("\n--- INICIANDO ENTRENAMIENTO DE LOS 5 MEJORES MODELOS ---")

for index, row in furiosos_5.iterrows():
    config = row['Configuracion']
    
    print(f"Entrenando modelo ID {row['ID']} con {config['num_neuronas_ocultas']} neuronas y LR {config['learning_rate']}...")
    
    # Crear el modelo con la configuración
    modelo = MLP_TODO(
        num_entradas=num_entradas,
        num_neuronas_ocultas=config['num_neuronas_ocultas'],
        num_salidas=1, # Clasificación binaria
        epochs=config['epochs'],
        batch_size=config['batch_size'],
        learning_rate=config['learning_rate'],
        initialization=config['inicializacion']
    )
    
    # Entrenar el modelo
    modelo.train(X_train_es, Y_train_es)
    
    # Almacenar el modelo y su ID para la gráfica
    lista_modelos_entrenados.append({
        'id_configuracion': row['ID'],
        'modelo': modelo,
        'f1_final': row['f1_score'] # F1-score obtenido en prueba
    })

print("--- ENTRENAMIENTO FINALIZADO ---")


--- INICIANDO ENTRENAMIENTO DE LOS 5 MEJORES MODELOS ---
Entrenando modelo ID 61 con 128 neuronas y LR 0.1...


TypeError: 'float' object cannot be interpreted as an integer

In [41]:
import matplotlib.pyplot as plt

def graficar_curva_f1_comparativa(lista_modelos_entrenados, ruta_guardado=None):
    """
    Genera una gráfica comparativa del F1-score vs. Época para los 5 mejores modelos.

    Args:
        lista_modelos_entrenados (list): Lista de diccionarios, cada uno con
                                        'modelo' (instancia de MLP_TODO) y 'f1_final'.
        ruta_guardado (str, optional): Ruta donde guardar la imagen. Si es None, solo la muestra.
    """
    plt.figure(figsize=(14, 8))
    
    colores = plt.cm.get_cmap('viridis', len(lista_modelos_entrenados))
    
    for i, item in enumerate(lista_modelos_entrenados):
        modelo = item['modelo']
        f1_final = item['f1_final']
        #config = modelo.configuracion # Asumiendo que guardaste la configuración en el modelo
        
        # Generar la etiqueta para la leyenda
        etiqueta = (
            i
        )
        
        # Verificar que el historial de métricas exista
        if hasattr(modelo, 'f1_score_epoca') and modelo.f1_score_epoca:
            # Los valores de F1-score se obtienen del atributo del modelo
            f1_scores = modelo.f1_score_epoca
            epochs = range(1, len(f1_scores) + 1)
            
            plt.plot(
                epochs, 
                f1_scores, 
                label=etiqueta, 
                color=colores(i),
                linewidth=2
            )
        else:
            print(f"Advertencia: El modelo ID {item['id_configuracion']} no tiene historial de F1-score.")

    # Títulos y Etiquetas
    plt.title('Curvas de Aprendizaje Comparativas (F1-score vs. Época) - Top 5 Modelos', fontsize=16)
    plt.xlabel('Época', fontsize=14)
    plt.ylabel('F1-score (Entrenamiento/Validación)', fontsize=14)
    plt.legend(title='Configuración', loc='lower right', fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    
    # Guardar o Mostrar
    if ruta_guardado:
        plt.savefig(ruta_guardado)
        print(f"\nGráfica guardada en: {ruta_guardado}")
        plt.close() # Cierra la figura para no mostrarla si se guarda
    else:
        plt.show()
    
    # La función devuelve la gráfica (aunque en Python es una operación, no un objeto matplotlib retornable)
    return ruta_guardado if ruta_guardado else "Gráfica mostrada en pantalla"

# --- EJECUTAR LA FUNCIÓN DE GRAFICACIÓN ---

# Define la ruta donde deseas guardar la gráfica
RUTA_GUARDADO = "./Resultados/curva_f1_top5_comparativa.svg" 

# Llamar a la función con los modelos entrenados
graficar_curva_f1_comparativa(lista_modelos_entrenados, ruta_guardado=RUTA_GUARDADO)


Gráfica guardada en: ./Resultados/curva_f1_top5_comparativa.svg


/var/folders/td/h2pqs96j2v5gv1p78d0qjz800000gn/T/ipykernel_1672/120306253.py:14: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colores = plt.cm.get_cmap('viridis', len(lista_modelos_entrenados))


'./Resultados/curva_f1_top5_comparativa.svg'